<a href="https://colab.research.google.com/github/ShuYuHuang/nlp4aml/blob/master/prep_5kdata_import.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('~/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /root/drive


In [ ]:
!pip install hanziconv
!pip install jieba

     |████████████████████████████████| 286kB 11.6MB/s 
  Created wheel for hanziconv: filename=hanziconv-0.3.2-py2.py3-none-any.whl size=23215 sha256=fe0038889da46bb69795855f846096b4e92b34b41a1187c2d78f7d585c9fdd0e
  Stored in directory: /root/.cache/pip/wheels/03/d8/3c/c39898fa9c9ce6e34b0ab4c6604892462d440c743715c94054
Successfully built hanziconv


In [ ]:
# coding=utf-8
import jieba 
from hanziconv import HanziConv
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [ ]:
file_loc='/root/drive/My Drive/ColabData/'
data_pd=pd.read_csv(file_loc+'tbrain_train_final_0610.csv')
def get_homepage(site):
  cnt=0
  begin=0
  for ee, ww in enumerate(site):
      if ww=='/':
        cnt+=1
        if cnt==2:
          begin=ee+1
        elif cnt==3:
          return site[begin:ee]
  return 0
data_pd['homepage']=data_pd['hyperlink'].apply(lambda w: get_homepage(w))

In [ ]:
data_pd.sample(5)

,news_ID,hyperlink,content,name,homepage
291,292,https://news.cnyes.com/news/id/4369409,0台股台股盤勢【華冠投顧】幸運星-全命中！！連三漲！！賺錢的能量持續加大中！！華冠投顧※來源...,[],news.cnyes.com
4551,4552,https://news.cnyes.com/news/id/4369175,A股港股大陸政經人行官員：降準備金率有空間但別期待太多鉅亨網編輯江泰傑2019/08/20 ...,[],news.cnyes.com
900,901,https://www.chinatimes.com/newspapers/20191125...,11月23日早晨 ### 省略內文 ### 讓社會更加幸福和諧。,[],www.chinatimes.com
2363,2364,https://www.bnext.com.tw/article/54760/digital...,在金融科技的趨勢下 ### 省略內文 ### 設計出金融服務的新樣貌。,[],www.bnext.com.tw
4857,4858,https://www.mirrormedia.mg/story/20190725cnt001/,2019.07.25 02:17頂新油案聲請提案大法庭魏應充：期待回歸理性 ### 省略內文...,[],www.mirrormedia.mg


In [ ]:
data_pd['homepage'].value_counts()

news.cnyes.com              709
www.mirrormedia.mg          626
www.chinatimes.com          621
m.ctee.com.tw               530
udn.com                     448
news.mingpao.com            379
news.ltn.com.tw             332
mops.twse.com.tw            273
technews.tw                 128
www.businesstoday.com.tw    120
www.ettoday.net             118
www.hk01.com                117
news.tvbs.com.tw            112
sina.com.hk                  72
www.bnext.com.tw             59
hk.on.cc                     57
finance.technews.tw          56
www.cw.com.tw                45
domestic.judicial.gov.tw     30
www.wealth.com.tw            29
tw.news.yahoo.com            27
www.coolloud.org.tw          25
www.setn.com                 16
www.hbrtaiwan.com            14
www.managertoday.com.tw      14
m.ltn.com.tw                 12
ccc.technews.tw              10
www.fsc.gov.tw                9
money.udn.com                 8
www.cna.com.tw                6
ec.ltn.com.tw                 6
ol.mingp

In [ ]:
display(data_pd[data_pd['homepage']=='sina.com.hk'][:3])
display(data_pd['hyperlink'][60])

,news_ID,hyperlink,content,name,homepage
60,61,https://sina.com.hk/news/article/20190919/1/11...,原標題：龐大董事換血 ### 省略內文 ### 增持計劃不設價格區間。新京報記者王琳琳圖片來...,[],sina.com.hk
203,204,https://sina.com.hk/news/article/20190927/1/11...,原標題：海馬、江淮汽車賣房補“窟窿” ### 省略內文 ### 不懼危機。新京報記者秦勝南編...,[],sina.com.hk
254,255,https://sina.com.hk/news/article/20190925/1/27...,原標題：特朗普遭眾院彈劾調查對手拜登支持度微升中新網9月25日電據“中央社”報導 ### 省...,[],sina.com.hk


'https://sina.com.hk/news/article/20190919/1/11/2/%E9%BE%90%E5%A4%A7%E8%91%A3%E4%BA%8B%E6%8F%9B%E8%A1%80%E6%B7%B1%E5%95%86%E6%8E%A7%E8%82%A1%E5%92%8C%E5%85%83%E7%B6%AD%E8%B3%87%E7%94%A2%E9%AB%98%E7%AE%A1%E5%85%A5%E9%A7%90-10637091.html'

In [ ]:
def extract_content(ii):
  global data_pd
  r = requests.get(data_pd['hyperlink'][ii])
  # Create soup from content of request
  c = r.content
  soup = BeautifulSoup(c)
  warns=False
  try:
    txt="<h1>"+soup.title.text+"</h1>"
    if data_pd['homepage'][ii]=='news.cnyes.com':
      txt+=soup.find('section').text
      txt+="<time>"+soup.find('time').text+"</time>"
  
    elif data_pd['homepage'][ii]=='www.mirrormedia.mg':
      for aa in soup.find_all('p'):
        txt+=aa.text
      txt+="<time>"+soup.find('div', attrs={"class":"date"}).text+"</time>"

    elif data_pd['homepage'][ii]=="www.chinatimes.com":
      txt+=soup.find('div', attrs={"class":"article-body"}).text
      txt+="<time>"+soup.find('time')['datetime']+"</time>"

    elif data_pd['homepage'][ii]=="m.ctee.com.tw":
      txt+=soup.find('div', attrs={"class":"entry-content"}).text
      txt+="<time>"+soup.find('div',attrs={"class":"post-meta-date"}).text+"</time>"

    elif data_pd['homepage'][ii]=="udn.com":
      txt+=soup.find('section',attrs={'class':'article-content__editor '}).text
      txt+="<time>"+soup.find('time').text+"</time>"

    elif data_pd['homepage'][ii]=="news.mingpao.com":
      txt+=soup.find('article').text
      txt+="<time>"+soup.find('div',attrs={'itemprop':'datePublished'}).text+"</time>"

    elif data_pd['homepage'][ii]=="news.ltn.com.tw":
      for aa in soup.find('div',attrs={'class':'text boxTitle boxText'}).find_all('p'):
        if not aa.has_attr('class'):
          txt+=aa.text
      txt+="<time>"+soup.find('span',attrs={'class':'time'}).text+"</time>"

    elif data_pd['homepage'][ii]=="mops.twse.com.tw":
      tmp=soup.find_all('td', attrs={"style":"text-align:left !important;"})
      txt="<h1>"+tmp[6].text+"</h1>"+tmp[9].text
      txt+="<time>"+tmp[1].text+"</time>"

    elif  data_pd['homepage'][ii]=="technews.tw":
      for aa in soup.find('div',class_="indent").find_all('p')[:-1]:
        txt+=aa.text
      txt+="<time>"+soup.find_all('span', attrs={"body"})[1].text+"</time>"

    elif  data_pd['homepage'][ii]=="www.businesstoday.com.tw":
      txt+=soup.find('div', attrs={"itemprop":"articleBody"}).text  
      txt+="<time>"+soup.find("meta",attrs={"property":"article:published_time"})['content']+"</time>"

    elif  data_pd['homepage'][ii]=="www.ettoday.net":
      for aa in soup.find("div",attrs={"class":"content-container"}).find_all("p")[3:]:
        if aa.text!='請繼續往下閱讀...':
          txt+=aa.text
      txt+="<time>"+soup.find("time",attrs={"class":"header-time"})['datetime']+"</time>" 

    elif data_pd['homepage'][ii]=='www.hk01.com':
      for aa in soup.findAll("p")[2:]:
        txt+=aa.text
      txt+="<time>"+soup.time.text+"</time>" 

    elif data_pd['homepage'][ii]=='news.tvbs.com.tw':
      for aa in AA.findAll('br')[::2]:
        txt+=aa.previous_element
      txt+="<time>"+soup.find("meta",attrs={"name":"pubdate"})["content"]+"</time>" 

    elif data_pd['homepage'][ii]=='sina.com.hk':
      for aa in soup.find("div",attrs={"class":"news-body"}).findAll("p")[:-5]:
        txt+=aa.text
      txt+="<time>"+soup.find("div",attrs={"class":"news-datetime"}).text+"</time>" 

    else:
      warns=True

    txt+="<aml>"+data_pd['name'][ii]+"</aml>"
    if warns==False:
      file1 = open(f"{file_loc}5k_news_data/{data_pd['news_ID'][ii]}.txt","w")#write mode 
      file1.write(txt) 
      file1.close()
  except:
    file1 = open(f"{file_loc}5k_news_data/{data_pd['news_ID'][ii]}.txt","w")#write mode 
    file1.write("404-error") 
    file1.close()

In [ ]:
for ii in data_pd.index:
  if ii>0:
    if data_pd['homepage'][ii]==data_pd['homepage'][ii-1]:
      time.sleep(1)
  extract_content(ii)